# Laboratório RAG com LangChain, Groq e Pinecone

Este notebook demonstra a implementação de um sistema Retrieval-Augmented Generation (RAG) utilizando a biblioteca LangChain, modelos de linguagem da Groq e o Pinecone como banco de dados vetorial. O objetivo é responder a perguntas com base em documentos PDF fornecidos, focando em atividades físicas e musculação.

## Configuração do Ambiente

### Importar bibliotecas

In [ ]:
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
import time
import json
from langchain_core.documents import Document
from langchain.chains.router import RouterChain, MultiPromptChain
from pydantic import BaseModel
from langchain.prompts import PromptTemplate
from pydantic.v1 import BaseModel



### Configuração das Chaves de API

In [ ]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

## Carregando a LLM da Groq

In [ ]:
llm = ChatGroq(model_name='llama-3.3-70b-versatile',temperature=0)

### Inicializar modelo de embeddings (HuggingFace para uso local/gratuito)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

## Carregamento e Processamento os dados em 'data'

In [ ]:
json_path = 'data/sample_places.json'

#Carregar o JSON com os dados
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

### Leitura dos dados que transforma os dados do documento em um documento LangChain

In [ ]:
documents = []
for item in data:
    content = f"{item['name']} ({item['city']}) - {item['type']}\n{item['description']}\nDicas: {item['tips']}"
    metadata = {"city": item["city"], "type": item["type"], "name": item["name"]}
    documents.append(Document(page_content=content, metadata=metadata))

print(f"Total de documentos: {len(documents)}")
print(documents[0])

## Geração de Embeddings e Armazenamento no Pinecone

### Divisão dos Documentos em Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


In [ ]:
print(f'Total de chunks: {len(docs)}')

In [ ]:
docs[1]

### Inicializar Pinecone

In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:
index_name = "rag-turismo" 
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

### Deletar e recriar o índice para garantir que a dimensão esteja correta

In [ ]:
if index_name in existing_indexes:
    print(f"Deletando o índice existente '{index_name}'...")
    pc.delete_index(index_name)
    time.sleep(1) # Aguardar a exclusão

In [ ]:
pc.create_index(
    name=index_name,
    metric="cosine",
    dimension=384,
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
while not pc.describe_index(index_name).status["ready"]:
    print("Aguardando o índice ficar pronto..."+pc.describe_index(index_name).status)
    time.sleep(1)

index = pc.Index(index_name)

### Criar ou conectar ao VectorStore do Pinecone

In [ ]:
vectorstore = PineconeVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    index_name=index_name,
)

## Construção da Cadeia RAG

#### Definir o prompt para a LLM

In [ ]:

prompt = ChatPromptTemplate.from_template("""Você é um assistente de turismo.
Responda à pergunta do usuário **somente** com base no contexto fornecido.
Se não souber, diga que não sabe e não invente.

Contexto: {context}

Pergunta: {input}""")

### Criar a cadeia de documentos

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)

### Criar a cadeia de recuperação

In [ ]:
retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), document_chain)

## Testando o Sistema RAG

In [ ]:
# Função para fazer perguntas
def ask_question(question):
    response = retrieval_chain.invoke({"input": question})
    print(f"Pergunta: {question}")
    print(f"Resposta: {response['answer']}")
    

In [ ]:
#Exemplos de perguntas (use os prompts e perguntas de teste )

pergunta1 = "Quais são os principais pontos turísticos do Rio de Janeiro?"

pergunta2 = "Dê dicas de segurança para quem vai visitar a Praia de Copacabana."

pergunta3 = "Como evitar filas para visitar a Torre Eiffel em Paris?"

pergunta4 = "O que posso ver no Museu do Louvre?"

pergunta5 = "Sugira um roteiro cultural de 3 dias em Paris."

pergunta6 = "Quais restaurantes ou dicas gastronômicas estão disponíveis no Rio de Janeiro?"

In [ ]:
ask_question(pergunta5)

### Testando com entrada interativa

In [ ]:
while True:
    question = input("\nDigite sua pergunta sobre turismo (ou 'sair' para encerrar): ")
    if question.lower() == "sair":
        break
    response = retrieval_chain.invoke({"input": question})
    print(f"Resposta: {response['answer']}")

## Criando as cadeias especialziadas

In [ ]:

from langchain.chains import LLMChain


In [ ]:
# Cadeias que usam RAG
itinerary_chain = retrieval_chain
logistics_chain = retrieval_chain
local_info_chain = retrieval_chain

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um guia de tradução para turistas. Responda com frases úteis e traduções claras."),
    ("human", "{input}")
])

translation_chain = LLMChain(llm=llm, prompt=chat_prompt)


### Criar Router Chain e MultiPromptChai

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Classifique a seguinte pergunta em uma das categorias:
- roteiro-viagem
- logistica-transporte
- info-local
- traducao-idiomas

Pergunta: {input}
Categoria:
""")


In [ ]:
router_chain = LLMChain(llm=llm, prompt=prompt)

default_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Desculpe, não entendi: {input}")
)



### Célula abaixo dando erro e não sendo possível resolver

In [ ]:
#multi_chain = MultiPromptChain(
#    router_chain=router_chain,
#    destination_chains={
#        "roteiro-viagem": itinerary_chain,
#        "logistica-transporte": logistics_chain,
#        "info-local": local_info_chain,
#        "traducao-idiomas": translation_chain,
#    },
#    default_chain=LLMChain(llm=llm, prompt=PromptTemplate.from_template("Desculpe, não entendi: {input}"))
#)


default_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Desculpe, não entendi: {input}")
)

destination_chains = {
    "roteiro-viagem": itinerary_chain,
    "logistica-transporte": logistics_chain,
    "info-local": local_info_chain,
    "traducao-idiomas": translation_chain,
}


# Roteador baseado em chave

router = RouterRunnable(
    get_destination=lambda input: input.get("categoria", ""),
    destinations=destination_chains,
    default=default_chain
)


# o erro acontece nesse RouterChain, onde ele não consegue resolver a importação de jeito nenhum.
multi_chain = RouterChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain
)




NameError: name 'LLMChain' is not defined

### Função de roteamento simples


In [ ]:

def route_input(user_input):
    if "viagem" in user_input.lower():
        return itinerary_chain.invoke({"input": user_input})
    elif "transporte" in user_input.lower():
        return logistics_chain.invoke({"input": user_input})
    elif "local" in user_input.lower():
        return local_info_chain.invoke({"input": user_input})
    elif "traduzir" in user_input.lower() or "idioma" in user_input.lower():
        return translation_chain.invoke({"input": user_input})
    else:
        return default_chain.invoke({"input": user_input})


### aqui é para garantir que a resposta seja tratada corretamente

In [ ]:
def extract_answer(response):
    if isinstance(response, str):
        return response
    elif isinstance(response, dict):
        # Tenta encontrar a chave mais provável
        for key in ["answer", "text", "result", "output"]:
            if key in response:
                return response[key]
        # Se não encontrar nenhuma chave conhecida, retorna tudo
        return str(response)
    elif hasattr(response, "content"):
        return response.content
    elif hasattr(response, "text"):
        return response.text
    else:
        return str(response)


### Função de entrada e loop interativo

In [ ]:
def process_question(question):
    response = route_input(question)
    answer = extract_answer(response)
    print(f"\nPergunta: {question}")
    print(f"Resposta: {answer}")


In [ ]:
# Testes iniciais
testes = [
    "Sugira um roteiro cultural de 3 dias em Paris.",
    "Como chegar ao Coliseu?",
    "Quais são os melhores restaurantes veganos em Tóquio?",
    "Como se diz 'onde fica o hotel?' em francês?"
]

for pergunta in testes:
    process_question(pergunta)

### Interface interativa

In [ ]:
while True:
    question = input("\nDigite sua pergunta sobre turismo (ou 'sair' para encerrar): ")
    if question.lower() == "sair":
        break
    process_question(question)


## Limpeza (Opcional)

In [ ]:
# Para deletar o índice do Pinecone (use com cautela!)
# pc.delete_index(index_name)